### Modules used in this notebook
xarray, numpy, cfgrib, regionmask, matplotlib

# C.005: how to apply a mask (country mask, weighted mask)

Load the packages

In [ ]:
import numpy as np
import cfgrib
import xarray as xr # xarray library access and manipulate multi-dimensional data (such as netcdf). With the cfgrib engine we can manipulate grib files
import regionmask # regionmask helps create of geographical regions.
from matplotlib import pyplot as plt # matplotlib package provides different functions to plot data

In [ ]:
# downloading the files from Goole Drive
# if you are running this notebook locally, please install gdown with `pip install gdown`
!gdown 1r-BSm3vF-Ey0ss-uZESU06u152ewsrGR

We use xarray to open the dataset

In [ ]:

ds = xr.open_dataset('era5-u100_v100_201903.grib')
ds = ds.assign({'ws100': (ds['u100']**2 + ds['v100']**2)**0.5})
ds

Select three domains:
- Germany
- France
-United Kingdom

Plot the hourly evolution of the wind speed at 100m averaged at the three domains

The outline of the countries are obtained from [Natural Earth](https://www.naturalearthdata.com/). They are automatically downloaded, cached and opened with geopandas.


In [ ]:
# create the cosine weight of latitudes
weigths = np.cos(np.deg2rad(ds.latitude))
# create mask of countries based on the data latitude and longitude
countries_masked = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.mask(ds, lon_name='longitude', lat_name='latitude')

countries_name = ['Germany', 'France', 'United Kingdom']
colors = ['blue', 'green', 'orange']

fig, ax = plt.subplots(figsize=(18, 12))
for ii, i in enumerate(countries_name):
    # select different countries
    ds_country =ds['ws100'].where(countries_masked==regionmask.defined_regions.natural_earth_v5_0_0.countries_110.map_keys(i))
    # applied the weighted mean over the country
    wnd100m_country = ds_country.weighted(weigths).mean(dim=('latitude', 'longitude'))
    ax.plot(wnd100m_country, color=colors[ii], label=i)
ax.set_xticks(np.arange(0, len(wnd100m_country.time), 3 * 24))
ax.set_xticklabels(wnd100m_country['time'].values[0: -1: 3 * 24].astype('datetime64[D]'), rotation=45, fontsize=12)
ax.set_ylim([0, 18])
ax.set_xlabel('Date', fontsize=14)
ax.set_ylabel('Wind Speed (m/s)', fontsize=14)
ax.legend(fontsize=15)
plt.title('Wind speed at 100m', fontsize=15)
plt.savefig('wind_speed_100m_countries.png')
